Data that looks like:

data = [
    {'closing_price': 102.06,
    'date': datetime.datetime(2014, 8, 29, 0, 0),
    'symbol': 'AAPL'},
    # ...
]

In [ ]:
# find max AAPL closing price with list comprehension
max_aapl_price = max(row["closing price"]
                    for row in data
                    if row["symbol"] == "AAPL")

In [ ]:
from collections import defaultdict

# group rows by symbol
by_symbol = defaultdict(list) # will insert empty list as value for any key that is accessed but not in dict
for row in data:
    by_symbol[row["symbol"]].append(row) # in the defaultdict, access the row's "symbol" value and
    # if the symbol value is in the dict, add the row to the dict as {row["symbol"] : [existing_row, new_row]}
    # otherwise create a new key: list-value entry in the default dict like {row["symbol"] : [new_row]}

# use a dict comprehension to find the max for each symbol
max_price_by_symbol = { symbol : max(row["closing_price"]                 # creates new dict with symbol : max price
                                    for row in grouped_rows)
                      for symbol, grouped_rows in by_symbol.iteritems() } # for each symbol and its associated list of
                                                                          # rows (daily stock prices)


In [4]:
def picker(field_name):
    """returns a function that picks a field out of a dict"""
    return lambda row: row[field_name] # so the output is a function that takes row as an arg but always accesses
                                       # the given field_name

def pluck(field_name, rows):
    """turn a list of dicts into the list of field_name values"""
    return map(picker(field_name), rows) # takes a list of row dicts, returns a list of picker(field_name)
                                         # applied to the elements in rows, i.e. a list of row[field_name]
# e.g.
import datetime    
sample_row = {'closing_price': 102.06,
    'date': datetime.datetime(2014, 8, 29, 0, 0),
    'symbol': 'AAPL'}

print picker("symbol")(sample_row)

AAPL


In [5]:
def group_by(grouper, rows, value_transform=None):
    # key is output of grouper, value is list of rows
    grouped = defaultdict(list)
    for row in rows:
        grouped[grouper(row)].append(row) #for each row, apply grouper to row and access as key; if present, append row
        # so say grouper sorts rows into types. Then grouped is a dict with keys = types and values = list of rows
        # of that type
    
    if value_transform is None:
        return grouped
    else:
        return {key: value_transform(rows)
               for key, rows in grouped.iteritems() } # so value_transform takes a list as an arg
    
# rewriting previous example:
max_price_by_symbol = group_by(picker("symbol"),
                              data,
                              lambda rows: max(pluck("closing_price", rows)))

<p> Now we ask, what are the largest and smallest one-day percent changes in our data set? <br>
Percent change: <br>
Today = (1 + percent change) * Yesterday OR <br>
Today / Yesterday - 1 = percent change </p>

In [1]:
def percent_price_change(yesterday, today):
    return today["closing_price"] / yesterday["closing_price"] - 1

def day_over_day_changes(grouped_rows): # grouped_rows is a list of dicts, each with a given symbol
    # sort the rows by date
    ordered = sorted(grouped_rows, key = picker("date")) # for each row dict in grouped_rows, sort on "date"-key value
    
    # zip with an offset to get pairs of consecutive days
    return [{ "symbol": today["symbol"],
              "date" : today["date"],
              "change" : percent_price_change(yesterday, today) }
           for yesterday, today in zip(ordered, ordered[1:])] # returns a dict for each pair of (yesterday, today)

# key is symbol, value is list of "change" dicts
changes_by_symbol = group_by(picker("symbol"), data, day_over_day_changes)
# this f'n first creates a dict with keys = symbols and values a list of row dicts. Then it applies
# day_over_day_changes to each symbol's list of row dicts, creating a list of 
# dicts, one for each symbol, of [{percent change dict 1}, ...]}

# collect all "change" dicts into one big list
all_changes = [change
              for changes in changes_by_symbol.values() # changes are [{percent change dict 1}, ...]
              for change in changes] # change is a single {percent change dict}

max(all_changes, key=picker("change")) # for each dict in all_changes, apply picker("change") (i.e. access the change
# value) and return the dict with the max "change" value